In [18]:
!pip install pandas dash
!pip install jupyter-dash
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2021-09-05 01:48:05--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3033 (3.0K) [text/csv]
Saving to: ‘spacex_launch_dash.csv.1’

spacex_launch_dash. 100%[===================>]   2.96K  --.-KB/s    in 0.001s  

2021-09-05 01:48:05 (4.32 MB/s) - ‘spacex_launch_dash.csv.1’ saved [3033/3033]



In [19]:
# Import required libraries
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd

In [20]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

locations = spacex_df.groupby(['Launch Site'], as_index=False).first()
locations_df = locations['Launch Site']

In [21]:
# Create a dash application
app = JupyterDash(__name__, external_stylesheets = external_stylesheets)
server = app.server

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                        # TASK 1: Add a dropdown list to enable Launch Site selection
                                        # The default select value is for ALL sites 
                                        # dcc.Dropdown(id='site-dropdown',...)
                                        
                                        dcc.Dropdown(id = 'site-dropdown',
                                        options = [{'label': 'All Sites', 'value': "All Sites"},
                                        {'label': locations_df[0], 'value': locations_df[0]},
                                        {'label': locations_df[1], 'value': locations_df[1]},
                                        {'label': locations_df[2], 'value': locations_df[2]},
                                        {'label': locations_df[3], 'value': locations_df[3]}],
                                        value = 'All Sites',
                                        placeholder= "Select a Launch Site here",
                                        searchable= True),
                                
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider', 
                                min = 0, max = 10000, 
                                step = 1000, 
                                marks={0: '0', 1000: '1000', 2000: '2000', 3000: '3000', 4000: '4000', 5000: '5000', 6000: '6000', 7000: '7000', 8000: '8000', 9000: '9000', 10000: '10000'}, 
                                value = [min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])


In [22]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

@app.callback(
    Output(component_id='success-pie-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'))

def pie_chart(site_dropdown):
    if  site_dropdown == 'All Sites':
        fig = px.pie(spacex_df, values='class', names='Launch Site', title= "Total Success by Site")
    else:
        title_pie = f"Success and Failure rate for site {site_dropdown}"
        filtered_DD= spacex_df[spacex_df['Launch Site'] == site_dropdown]
        filtered_LS = filtered_DD.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig = px.pie(filtered_LS, values='class count', names='class', title=title_pie) 
    return fig

In [23]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'), Input(component_id='payload-slider', component_property='value')])

def scatter_plot(site_dropdown,payload):

    low = payload[0]
    high = payload[1]

    filtered_df0 = spacex_df[spacex_df['Payload Mass (kg)'].between(low,high)]

    if site_dropdown == 'All Sites':
        fig = px.scatter(filtered_df0, x = 'Payload Mass (kg)', y = 'class', color="Booster Version Category", title = "Payload Vs Success for All Sites")
    else:
        filtered_Df1= filtered_df0[filtered_df0['Launch Site'] == site_dropdown]
        title = f"Payload vs Success for Site {site_dropdown}"
        fig = px.scatter(filtered_Df1, x = 'Payload Mass (kg)', y = 'class', color="Booster Version Category", title = title)
    return fig

In [24]:
# Run the app
app.run_server(mode="external", host = "127.0.0.1")

Dash app running on http://127.0.0.1:8050/
